# Instalação e importação de bibliotecas

In [ ]:
from google.colab import files
!pip install opendatasets -q U
import opendatasets as od
import pandas as pd
import glob
import os
# dataset URL
!pip install pandasql -q U
import pandasql as psql
# Ajustar para exibir todas as colunas
pd.set_option('display.max_columns', None)

  Preparing metadata (setup.py) ... done


# Coleta dos dados usando a API do Kaggle

Primeiro importamos um arquivo json com as informações da API do Kaggle

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"zemurillo","key":"4f2f4bbb6a17cd45586a49a7d090ebd9"}'}

Em seguida, podemos fazer o download dos dados.
Cada conjunto de dados possui uma série de dataframes, passaremos por cada um para encontrar as informações úteis para o projeto.

## WorldExpenditures

Este conjunto de dados refinado apresenta os padrões de gastos de 96 governos em 20 setores, do ano 2000 a 2021. As despesas são indicadas em milhões de dólares americanos, acompanhadas pelo GDP, que é a respectiva participação no PIB expressa em percentual.

source: https://unctadstat.unctad.org/datacentre/

In [ ]:
dataset = 'https://www.kaggle.com/datasets/adamgrey88/world-governments-expenditure-dataset-2000-2021'
# Using opendatasets let's download the data sets (480 MB)
od.download(dataset)

Dataset URL: https://www.kaggle.com/datasets/adamgrey88/world-governments-expenditure-dataset-2000-2021


100%|██████████| 436k/436k [00:00<00:00, 18.6MB/s]

In [ ]:
df_gov = pd.read_csv('world-governments-expenditure-dataset-2000-2021/WorldExpenditures.csv')
df_gov.head()

,Unnamed: 0,Year,Country,Sector,Expenditure(million USD),GDP(%)
0,0,2000,Australia,Total function,153122.633,37.36193
1,1,2000,Australia,"Agriculture, forestry, fishing and hunting",2195.583,0.53572
2,2,2000,Australia,"Mining, manufacturing and construction",905.018,0.22082
3,3,2000,Australia,Transport,11417.379,2.78584
4,4,2000,Australia,Fuel and energy,2251.241,0.54930


In [ ]:
#Excluindo coluna sem nome
df_gov.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
#Verificando todos os setores
df_gov.Sector.unique()

array(['Total function', 'Agriculture, forestry, fishing and hunting',
       'Mining, manufacturing and construction', 'Transport',
       'Fuel and energy', 'Environment protection',
       'Housing and community amenities', 'Health', 'Education',
       'Social protection', 'General public services', 'Defence',
       'Public order and safety', 'Recreation, culture and religion',
       'General economic, commercial and labour affairs',
       'Other industries', 'RandD Economic affairs',
       'Economic affairs n.e.c.', 'Communication', 'Others'], dtype=object)

In [ ]:
# Selecionando apenas os setores de interesse
df_gov = df_gov[(df_gov.Sector == 'Health') | (df_gov.Sector =='Education') |(df_gov.Sector == 'Recreation, culture and religion')]

In [ ]:
df_gov.head()

,Year,Country,Sector,Expenditure(million USD),GDP(%)
7,2000,Australia,Health,24071.984,5.87357
8,2000,Australia,Education,23066.666,5.62827
13,2000,Australia,"Recreation, culture and religion",4096.644,0.99958
26,2000,Austria,Health,14053.707,7.14112
27,2000,Austria,Education,10269.251,5.21813


Realizando uma estimativa simples dos anos ausentes duplicando a linha do último ano para anos futuros.

In [ ]:
def fill_missing_years(df):
    # Encontrar o último ano disponível para cada país e setor
    last_years = df.groupby(['Country', 'Sector'])['Year'].max().reset_index()
    last_years = last_years.rename(columns={'Year': 'LastYear'})

    # Mesclar para obter os dados do último ano
    df = pd.merge(df, last_years, on=['Country', 'Sector'])

    # Obter os dados do último ano
    last_year_data = df[df['Year'] == df['LastYear']].drop(columns='LastYear')

    # Criar uma lista de anos para adicionar
    years_to_add = [2021, 2022, 2023]

    # Adicionar dados para os anos faltantes
    new_rows = []
    for _, row in last_year_data.iterrows():
        for year in years_to_add:
            if year > row['Year']:
                new_row = row.copy()
                new_row['Year'] = year
                new_rows.append(new_row)

    # Adicionar novas linhas ao DataFrame
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

    # Ordenar o DataFrame por país, setor e ano
    df = df.sort_values(by=['Country', 'Sector', 'Year']).reset_index(drop=True)

    return df

# Preencher os anos ausentes
df_gov_filled = fill_missing_years(df_gov)

In [ ]:
df_gov_filled.head()

,Year,Country,Sector,Expenditure(million USD),GDP(%),LastYear
0,2006,Afghanistan,Education,324.499,4.56737,2017.0
1,2007,Afghanistan,Education,357.221,3.79531,2017.0
2,2008,Afghanistan,Education,451.464,4.41017,2017.0
3,2009,Afghanistan,Education,580.457,5.00604,2017.0
4,2010,Afghanistan,Education,824.899,5.61198,2017.0


In [ ]:
df_gov_filled[(df_gov_filled.Country == 'China') & (df_gov_filled.Sector == 'Recreation, culture and religion') ]

,Year,Country,Sector,Expenditure(million USD),GDP(%),LastYear
740,2005,China,"Recreation, culture and religion",7731.011,0.33820,2020.0
741,2006,China,"Recreation, culture and religion",9164.177,0.33299,2020.0
742,2007,China,"Recreation, culture and religion",12400.947,0.34929,2020.0
743,2008,China,"Recreation, culture and religion",21182.085,0.46105,2020.0
744,2009,China,"Recreation, culture and religion",25815.585,0.50602,2020.0
745,2010,China,"Recreation, culture and religion",29104.456,0.47813,2020.0
746,2011,China,"Recreation, culture and religion",32486.924,0.43020,2020.0
747,2012,China,"Recreation, culture and religion",37949.837,0.44478,2020.0
748,2013,China,"Recreation, culture and religion",43526.068,0.45480,2020.0
749,2014,China,"Recreation, culture and religion",46542.047,0.44429,2020.0


In [ ]:
df_gov_filled[(df_gov_filled.Country == 'Brazil') & (df_gov_filled.Sector == 'Education') ]

,Year,Country,Sector,Expenditure(million USD),GDP(%),LastYear
477,2000,Brazil,Education,30618.750,4.67142,2020.0
478,2001,Brazil,Education,26850.987,4.79496,2020.0
479,2002,Brazil,Education,19256.640,3.77733,2020.0
480,2003,Brazil,Education,26254.828,4.70320,2020.0
481,2004,Brazil,Education,27586.682,4.12179,2020.0
482,2005,Brazil,Education,36276.648,4.06856,2020.0
483,2006,Brazil,Education,46621.977,4.20918,2020.0
484,2007,Brazil,Education,63190.480,4.52293,2020.0
485,2008,Brazil,Education,79667.415,4.69777,2020.0
486,2009,Brazil,Education,84619.439,5.07616,2020.0


In [ ]:
df_gov = df_gov_filled

## Olympic Summer & Winter Games, 1896-2022 Dataset

O conjunto de dados contém informações sobre medalhas, resultados e atletas. Mais de 21.000 medalhas, 162.000 resultados, 74.000 atletas, 20.000 biografias e os anfitriões dos Jogos Olímpicos de Verão e de Inverno podem ser encontrados aqui.

In [ ]:
dataset = 'https://www.kaggle.com/datasets/piterfm/olympic-games-medals-19862018'
od.download(dataset)

Dataset URL: https://www.kaggle.com/datasets/piterfm/olympic-games-medals-19862018


100%|██████████| 13.9M/13.9M [00:00<00:00, 93.3MB/s]

### Atletas

In [ ]:
df_atletas = pd.read_csv('olympic-games-medals-19862018/olympic_athletes.csv')
df_atletas.head()

,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,athlete_medals,bio
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000.0,NaN,NaN
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995.0,NaN,NaN
2,https://olympics.com/en/athletes/dylan-walczyk,Dylan WALCZYK,1,Beijing 2022,1993.0,NaN,NaN
3,https://olympics.com/en/athletes/olli-penttala,Olli PENTTALA,1,Beijing 2022,1995.0,NaN,NaN
4,https://olympics.com/en/athletes/reikherd,Dmitriy REIKHERD,1,Beijing 2022,1989.0,NaN,NaN


### Hosts

In [ ]:
df_hosts = pd.read_csv('olympic-games-medals-19862018/olympic_hosts.csv')
df_hosts.head()

,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014


### Medalhas

Medalhas por atleta

In [ ]:
df_medalhas = pd.read_csv('olympic-games-medals-19862018/olympic_medals.csv')
df_medalhas.head(2)

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,Italy,IT,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,Italy,IT,ITA


### Resultados

In [ ]:
df_resultados = pd.read_csv('olympic-games-medals-19862018/olympic_results.csv')
df_resultados.head(2)

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,NaN,NaN,NaN,NaN
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,NaN,NaN,NaN,NaN


## Paris 2024 Olympic Summer Games

O conjunto de dados dos Jogos Olímpicos de Verão de Paris 2024 fornece informações abrangentes sobre os Jogos Olímpicos realizados em 2024. Ele abrange vários aspectos do evento, incluindo países participantes, atletas, disciplinas esportivas, classificação de medalhas e detalhes importantes do evento.

https://www.kaggle.com/datasets/piterfm/paris-2024-olympic-summer-games



In [ ]:
dataset = 'https://www.kaggle.com/datasets/piterfm/paris-2024-olympic-summer-games'

In [ ]:
od.download(dataset, force = True)

Dataset URL: https://www.kaggle.com/datasets/piterfm/paris-2024-olympic-summer-games


100%|██████████| 2.95M/2.95M [00:00<00:00, 40.3MB/s]

### Resultados

In [ ]:
df_basquete_2024 = pd.read_csv('paris-2024-olympic-summer-games/results/3x3 Basketball.csv')

In [ ]:
df_basquete_2024.head(2)

,date,stage_code,event_code,event_name,event_stage,stage,gender,discipline_name,discipline_code,venue,participant_code,participant_name,participant_type,participant_country_code,participant_country,result,result_type,result_WLT,start_order
0,2024-07-30T18:59:05+02:00,BK3MTEAM3-------------GPA-000100--,BK3MTEAM3,Men,Men's Pool Round,Pool Round,M,3x3 Basketball,BK3,La Concorde 1,BK3MTEAM3---LAT01,Latvia,Team,LAT,Latvia,21,POINTS,W,1
1,2024-07-30T18:59:05+02:00,BK3MTEAM3-------------GPA-000100--,BK3MTEAM3,Men,Men's Pool Round,Pool Round,M,3x3 Basketball,BK3,La Concorde 1,BK3MTEAM3---LTU01,Lithuania,Team,LTU,Lithuania,14,POINTS,L,2


Os resultados estão divididos por esporte. Precisaremos concatenar todos os resultados em um único dataframe.

In [ ]:
# Caminho para a pasta que contém os arquivos CSV
caminho_pasta = "/content/paris-2024-olympic-summer-games/results/"

# Lista para armazenar cada DataFrame
lista_dfs = []

# Loop para percorrer todos os arquivos na pasta
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith(".csv"):
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        df = pd.read_csv(caminho_arquivo)
        lista_dfs.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_resultados_2024 = pd.concat(lista_dfs, ignore_index=True)


In [ ]:
df_resultados_2024.head(1)

,date,stage_code,event_code,event_name,event_stage,stage,gender,discipline_name,discipline_code,venue,participant_code,participant_name,participant_type,participant_country_code,participant_country,rank,result,result_type,result_IRM,result_diff,qualification_mark,start_order,bib,result_WLT
0,2024-07-28T12:09:19+02:00,ROWMSCULL2-L----------HEAT000100--,ROWMSCULL2-L,Lightweight Men's Double Sculls,Lightweight Men's Double Sculls Heat 1,Heat 1,M,Rowing,ROW,Nautical St - Flat water,ROWMSCULL2-LSUI01,Switzerland,Team,SUI,Switzerland,1.0,6:24.88,TIME,NaN,NaN,SA/B,4,NaN,NaN


In [ ]:
df_resultados_2024.drop_duplicates(inplace=True)

### Atletas

In [ ]:
df_atletas2024 = pd.read_csv('paris-2024-olympic-summer-games/athletes.csv')

In [ ]:
df_atletas2024.head(2)

,code,current,name,name_short,name_tv,gender,function,country_code,country,country_long,nationality,nationality_long,nationality_code,height,weight,disciplines,events,birth_date,birth_place,birth_country,residence_place,residence_country,nickname,hobbies,occupation,education,family,lang,coach,reason,hero,influence,philosophy,sporting_relatives,ritual,other_sports
0,1532872,True,ALEKSANYAN Artur,ALEKSANYAN A,Artur ALEKSANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0.0,0.0,['Wrestling'],"[""Men's Greco-Roman 97kg""]",1991-10-21,GYUMRI,Armenia,GYUMRI,Armenia,White Bear,Playing and watching football,Athlete,Graduated from Shirak State University (Gyumri...,"Father, Gevorg Aleksanyan","Armenian, English, Russian","Gevorg Aleksanyan (ARM), father",He followed his father and his uncle into the ...,"Footballer Zinedine Zidane (FRA), World Cup wi...","His father, Gevorg Aleksanyan","""Wrestling is my life."" (mediamax.am. 18 May 2...",NaN,NaN,NaN
1,1532873,True,AMOYAN Malkhas,AMOYAN M,Malkhas AMOYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0.0,0.0,['Wrestling'],"[""Men's Greco-Roman 77kg""]",1999-01-22,YEREVAN,Armenia,YEREVAN,Armenia,NaN,NaN,NaN,NaN,NaN,Armenian,NaN,NaN,NaN,NaN,"""To become a good athlete, you first have to b...","Uncle, Roman Amoyan (wrestling), 2008 Olympic ...",NaN,NaN


### Medalhas

In [ ]:
df_medalhas2024 = pd.read_csv('paris-2024-olympic-summer-games/medals.csv')

In [ ]:
df_medalhas2024.head(2)

,medal_type,medal_code,medal_date,name,gender,discipline,event,event_type,url_event,code,country_code,country,country_long
0,Gold Medal,1.0,2024-07-27,Remco EVENEPOEL,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1903136,BEL,Belgium,Belgium
1,Silver Medal,2.0,2024-07-27,Filippo GANNA,M,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1923520,ITA,Italy,Italy


In [ ]:
df_medalhas2024.medal_type.unique()

array(['Gold Medal', 'Silver Medal', 'Bronze Medal'], dtype=object)

### Medalhistas

In [ ]:
df_medalhistas2024 = pd.read_csv('paris-2024-olympic-summer-games/medallists.csv')

In [ ]:
df_medalhistas2024.head(2)

,medal_date,medal_type,medal_code,name,gender,country_code,country,country_long,nationality_code,nationality,nationality_long,team,team_gender,discipline,event,event_type,url_event,birth_date,code_athlete,code_team,is_medallist
0,2024-07-27,Gold Medal,1.0,EVENEPOEL Remco,Male,BEL,Belgium,Belgium,BEL,Belgium,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,2000-01-25,1903136,NaN,True
1,2024-07-27,Silver Medal,2.0,GANNA Filippo,Male,ITA,Italy,Italy,ITA,Italy,Italy,NaN,NaN,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1996-07-25,1923520,NaN,True


### Medalhas - Total

In [ ]:
df_medalhas_2024 = pd.read_csv('paris-2024-olympic-summer-games/medals_total.csv')

In [ ]:
df_medalhas_2024.head()

,country_code,country,country_long,Gold Medal,Silver Medal,Bronze Medal,Total
0,USA,United States,United States of America,40,44,42,126
1,CHN,China,People's Republic of China,40,27,24,91
2,JPN,Japan,Japan,20,12,13,45
3,AUS,Australia,Australia,18,19,16,53
4,FRA,France,France,16,26,22,64


### Times

In [ ]:
df_times2024 = pd.read_csv('paris-2024-olympic-summer-games/teams.csv')

In [ ]:
df_times2024.head(2)

,code,current,team,team_gender,country_code,country,country_long,discipline,disciplines_code,events,athletes,coaches,athletes_codes,num_athletes,coaches_codes,num_coaches
0,ARCMTEAM3---CHN01,True,People's Republic of China,M,CHN,China,People's Republic of China,Archery,ARC,Men's Team,"['KAO Wenchao', 'LI Zhongyuan', 'WANG Yan']",NaN,"['1913366', '1913367', '1913369']",3.0,NaN,NaN
1,ARCMTEAM3---COL01,True,Colombia,M,COL,Colombia,Colombia,Archery,ARC,Men's Team,"['ARCILA Santiago', 'ENRIQUEZ Jorge', 'HERNAND...",NaN,"['1935642', '1543412', '1935644']",3.0,NaN,NaN


# Preparação dos dados / Engenharia de dados

Nessa etapa, iremos tratar os dados e realizar cruzamentos para chegar no dataframe final que irá alimentar a nossa regressão linear. Como temos um conjunto de dados para as olimpíadas de 2024, e outro conjunto para olimpíadas anteriores, teremos que realizar tratamentos separados.



**Variáveis** para a regressão linear:

* **Qtd_Medalhas_Pais:** Essa é a variável target, que queremos prever
* **Qtd_Esportes_Distintos:** Quantidade de esportes distintos que um país participou nas olimpíadas.
* **Qtd_Times**: Essa variável expressa o total de times que o país teve em todas as competições. Competidores individuais também são considerados como um time e caso o mesmo competidor compita em duas ou mais modalidades diferentes, ele também será contado como dois ou mais times diferentes.
* **Health_Expenditure:** Soma dos investimentos em saúde do país nos 3 anos anteriores a olimpíadas.
* **Health_Media_Gpd:** Média do Gpd dos investimentos em saúde do país nos 3 anos anteriores a olimpíadas.
* **Culture_Expenditure:** Soma dos investimentos em cultura do país nos 3 anos anteriores a olimpíada
* **Culture_Media_Gpd:** Média do Gpd dos investimentos em cultura do país nos 3 anos anteriores a olimpíadas.
* **Education_Expenditure:** Soma dos investimentos em educação do país nos 3 anos anteriores a olimpíada.
* **Education_Media_Gpd:** Média do Gpd dos investimentos em educação do país nos 3 anos anteriores a olimpíadas.

A tratamento dos dados foca em manipular os dados para chegar nessas variáveis por país e edição das olimpíadas. Como só temos dados socioecômicos de 2000 a 2021, iremos trabalhar com os dados das olimpíadas de 2004 a 2024.




## Preparação dos dados das olimpíadas de 2004 a 2020

In [ ]:
df_hosts.head(2)

,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020


Retiraremos os resultados das Olimpíadas de Inverno, pois o desempenho dos países varia significativamente quando comparamos as Olimpíadas de Inverno com as de Verão. Nosso objetivo é prever o resultado das Olimpíadas de Verão em 2024.

In [ ]:
df_resultados = pd.merge(df_resultados,df_hosts[['game_slug', 'game_season','game_year']], left_on = 'slug_game',right_on = 'game_slug', how = 'left')
df_resultados.head(2)

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type,game_slug,game_season,game_year
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,NaN,NaN,NaN,NaN,beijing-2022,Winter,2022
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,NaN,NaN,NaN,NaN,beijing-2022,Winter,2022


In [ ]:
#Retirando jogos de inverno
df_resultados = df_resultados[df_resultados['game_season'] == 'Summer']

#Excluindo colunas duplicadas
df_resultados.drop(columns = ['game_slug'],inplace = True)

In [ ]:
df_resultados['game_season'].unique()

array(['Summer'], dtype=object)

In [ ]:
# Retirando onde não tem medalha
#df_resultados = df_resultados.dropna(subset=['medal_type'])

#Retirando jogos anteriores a 2000
df_resultados = df_resultados[df_resultados['game_year'] >= 2000]

df_resultados.head(2)

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type,game_season,game_year
3784,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,GOLD,"[('Fatima GALVEZ', 'https://olympics.com/en/at...",False,1,Spain,ES,ESP,NaN,NaN,NaN,NaN,Summer,2020
3785,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,SILVER,"[('Alessandra PERILLI', 'https://olympics.com/...",False,2,San Marino,SM,SMR,NaN,NaN,NaN,NaN,Summer,2020


In [ ]:
df_medalhas_pais_tipo = df_resultados.dropna(subset=['medal_type'])
df_medalhas_pais_tipo = df_medalhas_pais_tipo.groupby(['slug_game','medal_type','country_name','country_code','country_3_letter_code','game_year']).size().reset_index(name='Qtd_Medalhas_Pais')
df_medalhas_pais_tipo.head()

,slug_game,medal_type,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais
0,athens-2004,BRONZE,Argentina,AR,ARG,2004,4
1,athens-2004,BRONZE,Australia,AU,AUS,2004,17
2,athens-2004,BRONZE,Austria,AT,AUT,2004,1
3,athens-2004,BRONZE,Azerbaijan,AZ,AZE,2004,4
4,athens-2004,BRONZE,Bahamas,BS,BAH,2004,1


In [ ]:
# Primeiro dropa o que não for medalha
# df_medalhas_pais = df_resultados.dropna(subset=['medal_type'])
# df_medalhas_pais = df_medalhas_pais.groupby(['slug_game','country_name','country_code','country_3_letter_code','game_year']).size().reset_index(name='Qtd_Medalhas_Pais')
# df_medalhas_pais.head()

df_medalhas_pais = df_resultados.dropna(subset=['medal_type'])
df_medalhas_pais = df_medalhas_pais.groupby(['slug_game','medal_type','country_name','country_code','country_3_letter_code','game_year']).size().reset_index(name='Qtd_Medalhas_Pais')
df_medalhas_pais.head()

,slug_game,medal_type,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais
0,athens-2004,BRONZE,Argentina,AR,ARG,2004,4
1,athens-2004,BRONZE,Australia,AU,AUS,2004,17
2,athens-2004,BRONZE,Austria,AT,AUT,2004,1
3,athens-2004,BRONZE,Azerbaijan,AZ,AZE,2004,4
4,athens-2004,BRONZE,Bahamas,BS,BAH,2004,1


In [ ]:
df_medalhas_pais.medal_type.unique()

array(['BRONZE', 'GOLD', 'SILVER'], dtype=object)

In [ ]:
query = """
select
     slug_game
    ,country_name
    ,country_code
    ,country_3_letter_code
    ,game_year
    ,sum(Qtd_Medalhas_Pais)   as Qtd_Medalhas_Pais
    ,sum(Qtd_Medalhas_Ouro)   as Qtd_Medalhas_Ouro
    ,sum(Qtd_Medalhas_Prata)  as Qtd_Medalhas_Prata
    ,sum(Qtd_Medalhas_Bronze) as Qtd_Medalhas_Bronze
from (
  select
     slug_game
    ,country_name
    ,country_code
    ,country_3_letter_code
    ,game_year
    ,Qtd_Medalhas_Pais
    ,case when medal_type = 'GOLD' then Qtd_Medalhas_Pais   else 0 end as Qtd_Medalhas_Ouro
    ,case when medal_type = 'SILVER' then Qtd_Medalhas_Pais else 0 end as Qtd_Medalhas_Prata
    ,case when medal_type = 'BRONZE' then Qtd_Medalhas_Pais else 0 end as Qtd_Medalhas_Bronze
  from df_medalhas_pais
  )
  group by
     slug_game
    ,country_name
    ,country_code
    ,country_3_letter_code
    ,game_year
"""
df_medalhas_pais = psql.sqldf(query, locals())

In [ ]:
df_medalhas_pais.head()

,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze
0,athens-2004,Argentina,AR,ARG,2004,6,2,0,4
1,athens-2004,Australia,AU,AUS,2004,50,17,16,17
2,athens-2004,Austria,AT,AUT,2004,7,2,4,1
3,athens-2004,Azerbaijan,AZ,AZE,2004,5,1,0,4
4,athens-2004,Bahamas,BS,BAH,2004,2,1,0,1


Trazendo informações de quantidade de madalhas em esportes individuais

In [ ]:
df_medalhas_solo = df_resultados.groupby(['participant_type','slug_game','country_name','country_code','country_3_letter_code','game_year']).size().reset_index(name='Qtd_Medalhas_Solo')
df_medalhas_solo = df_medalhas_solo[df_medalhas_solo['participant_type'] == 'Athlete']
df_medalhas_solo.head()

,participant_type,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Solo
0,Athlete,athens-2004,Afghanistan,AF,AFG,2004,2
1,Athlete,athens-2004,Albania,AL,ALB,2004,5
2,Athlete,athens-2004,Algeria,DZ,ALG,2004,45
3,Athlete,athens-2004,American Samoa,AS,ASA,2004,1
4,Athlete,athens-2004,Andorra,AD,AND,2004,4


Quantidade de times (individuais ou em grupos) por esporte + categoria que o país teve representando o país na edição. No fim, é uma boa proxy para a quantidade de atletas que ele está levando.

Exemplo: Judo --> 6 atletas participando de 8 modalidades (chance de 8 medalhas).



In [ ]:
# Quantidade de times por categoria e esporte do país
df_qtdTimes_por_Caterogia = df_resultados.groupby(['slug_game','discipline_title','event_title','participant_type','country_name','country_code','country_3_letter_code','game_year']).size().reset_index(name='Qtd_Times')
df_qtdTimes_por_Caterogia.head(2)

,slug_game,discipline_title,event_title,participant_type,country_name,country_code,country_3_letter_code,game_year,Qtd_Times
0,athens-2004,Archery,Individual competition men,Athlete,Australia,AU,AUS,2004,3
1,athens-2004,Archery,Individual competition men,Athlete,Belarus,BY,BLR,2004,1


In [ ]:
#Total de times (individuais ou em grupos) que cada país tem em todos os esportes + categoias)
df_times = df_qtdTimes_por_Caterogia.groupby(['slug_game','game_year','country_name','country_code','country_3_letter_code'])['Qtd_Times'].sum().reset_index(name='Qtd_Times')

In [ ]:
df_times.info() # 1143  linhas, enquanto o df_resultados completo tem 43496 linhas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   slug_game              1143 non-null   object
 1   game_year              1143 non-null   int64 
 2   country_name           1143 non-null   object
 3   country_code           1143 non-null   object
 4   country_3_letter_code  1143 non-null   object
 5   Qtd_Times              1143 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 53.7+ KB


Quantidade de esportes distintos por país e edição

In [ ]:
df_qtd_esportes = df_resultados.groupby(['slug_game','game_year','country_name','country_code','country_3_letter_code'])['discipline_title'].nunique().reset_index(name='Qtd_Esportes_Distintos')
df_qtd_esportes.head()

,slug_game,game_year,country_name,country_code,country_3_letter_code,Qtd_Esportes_Distintos
0,athens-2004,2004,Afghanistan,AF,AFG,2
1,athens-2004,2004,Albania,AL,ALB,3
2,athens-2004,2004,Algeria,DZ,ALG,10
3,athens-2004,2004,American Samoa,AS,ASA,1
4,athens-2004,2004,Andorra,AD,AND,3


### Atletas

In [ ]:
df_atletas.head()

,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,athlete_medals,bio
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000.0,NaN,NaN
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995.0,NaN,NaN
2,https://olympics.com/en/athletes/dylan-walczyk,Dylan WALCZYK,1,Beijing 2022,1993.0,NaN,NaN
3,https://olympics.com/en/athletes/olli-penttala,Olli PENTTALA,1,Beijing 2022,1995.0,NaN,NaN
4,https://olympics.com/en/athletes/reikherd,Dmitriy REIKHERD,1,Beijing 2022,1989.0,NaN,NaN


In [ ]:
df_medalhas.head()

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,Italy,IT,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,Italy,IT,ITA
2,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,Norway,NO,NOR
3,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,Norway,NO,NOR
4,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,Sweden,SE,SWE


In [ ]:
df_medalhas = pd.merge(df_medalhas,df_hosts[['game_slug', 'game_season','game_year']], left_on = 'slug_game',right_on = 'game_slug', how = 'left')

In [ ]:
#Retirando jogos de inverno
df_medalhas = df_medalhas[df_medalhas['game_season'] == 'Summer']

#Excluindo colunas duplicadas
df_medalhas.drop(columns = ['game_slug'],inplace = True)

In [ ]:
#Retirando jogos anteriores a 2000
df_medalhas = df_medalhas[df_medalhas['game_year'] >= 2000]

In [ ]:
df_medalhas.head()

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code,game_season,game_year
355,Shooting,tokyo-2020,Trap Mixed Team,Mixed,GOLD,GameTeam,Spain,https://olympics.com/en/athletes/fatima-galvez,Fatima GALVEZ,Spain,ES,ESP,Summer,2020
356,Shooting,tokyo-2020,Trap Mixed Team,Mixed,GOLD,GameTeam,Spain,https://olympics.com/en/athletes/alberto-ferna...,Alberto FERNANDEZ,Spain,ES,ESP,Summer,2020
357,Shooting,tokyo-2020,Trap Mixed Team,Mixed,SILVER,GameTeam,San Marino,https://olympics.com/en/athletes/alessandra-pe...,Alessandra PERILLI,San Marino,SM,SMR,Summer,2020
358,Shooting,tokyo-2020,Trap Mixed Team,Mixed,SILVER,GameTeam,San Marino,https://olympics.com/en/athletes/gian-marco-berti,Gian Marco BERTI,San Marino,SM,SMR,Summer,2020
359,Shooting,tokyo-2020,Trap Mixed Team,Mixed,BRONZE,GameTeam,United States of America,https://olympics.com/en/athletes/madelynn-ann-...,Madelynn Ann BERNAU,United States of America,US,USA,Summer,2020


In [ ]:
df_medalhas.participant_type.unique()

array(['GameTeam', 'Athlete'], dtype=object)

In [ ]:
df_medalhas_atletas = df_medalhas[df_medalhas['participant_type'] =='Athlete'].groupby(['slug_game','game_year','athlete_url','country_name','country_code','country_3_letter_code']).size().reset_index(name='Qtd_Medalhas_Individuais')
df_medalhas_atletas['ano_olimpíada_anterior'] = df_medalhas_atletas['game_year'] - 4
df_medalhas_atletas['ano_proxima_olimpíada'] = df_medalhas_atletas['game_year'] + 4
df_medalhas_atletas.head()


,slug_game,game_year,athlete_url,country_name,country_code,country_3_letter_code,Qtd_Medalhas_Individuais,ano_olimpiada_anterior,ano_proxima_olimpiada
0,athens-2004,2004,https://olympics.com/en/athletes/aaron-peirsol,United States of America,US,USA,2,2000,2008
1,athens-2004,2004,https://olympics.com/en/athletes/adam-nelson,United States of America,US,USA,1,2000,2008
2,athens-2004,2004,https://olympics.com/en/athletes/adam-van-koev...,Canada,CA,CAN,2,2000,2008
3,athens-2004,2004,https://olympics.com/en/athletes/adam-vella,Australia,AU,AUS,1,2000,2008
4,athens-2004,2004,https://olympics.com/en/athletes/adriana-carmo...,Venezuela,VE,VEN,1,2000,2008


Trazer se o atleta foi medalhista na olimpíada anterior
usando queries SQL para isso:

In [ ]:
query = """
select
   a.*
  ,case when b.athlete_url is null then 0 else 1 end as flag_medalhista_olimpíada_ant
from df_medalhas_atletas as a
left join df_medalhas_atletas as b on a.athlete_url = b.athlete_url and a.ano_olimpíada_anterior = b.game_year
"""
df_medalhistas_omplipadas_ant = psql.sqldf(query, locals())

In [ ]:
print(len(df_medalhas_atletas))
print(len(df_medalhistas_omplipadas_ant))

3899
3899


In [ ]:
df_medalhistas_omplipadas_ant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3899 entries, 0 to 3898
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   slug_game                      3899 non-null   object
 1   game_year                      3899 non-null   int64 
 2   athlete_url                    3899 non-null   object
 3   country_name                   3899 non-null   object
 4   country_code                   3899 non-null   object
 5   country_3_letter_code          3899 non-null   object
 6   Qtd_Medalhas_Individuais       3899 non-null   int64 
 7   ano_olimpiada_anterior         3899 non-null   int64 
 8   ano_proxima_olimpiada          3899 non-null   int64 
 9   flag_medalhista_olimpiada_ant  3899 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 304.7+ KB


In [ ]:
df_medalhistas_omplipadas_ant.head()

,slug_game,game_year,athlete_url,country_name,country_code,country_3_letter_code,Qtd_Medalhas_Individuais,ano_olimpiada_anterior,ano_proxima_olimpiada,flag_medalhista_olimpiada_ant
0,athens-2004,2004,https://olympics.com/en/athletes/aaron-peirsol,United States of America,US,USA,2,2000,2008,1
1,athens-2004,2004,https://olympics.com/en/athletes/adam-nelson,United States of America,US,USA,1,2000,2008,1
2,athens-2004,2004,https://olympics.com/en/athletes/adam-van-koev...,Canada,CA,CAN,2,2000,2008,0
3,athens-2004,2004,https://olympics.com/en/athletes/adam-vella,Australia,AU,AUS,1,2000,2008,0
4,athens-2004,2004,https://olympics.com/en/athletes/adriana-carmo...,Venezuela,VE,VEN,1,2000,2008,0


In [ ]:
df_medalhistas_omplipadas_ant[df_medalhistas_omplipadas_ant.athlete_url == 'https://olympics.com/en/athletes/aaron-peirsol']

,slug_game,game_year,athlete_url,country_name,country_code,country_3_letter_code,Qtd_Medalhas_Individuais,ano_olimpiada_anterior,ano_proxima_olimpiada,flag_medalhista_olimpiada_ant
0,athens-2004,2004,https://olympics.com/en/athletes/aaron-peirsol,United States of America,US,USA,2,2000,2008,1
601,beijing-2008,2008,https://olympics.com/en/athletes/aaron-peirsol,United States of America,US,USA,2,2004,2012,1
2557,sydney-2000,2000,https://olympics.com/en/athletes/aaron-peirsol,United States of America,US,USA,1,1996,2004,0


In [ ]:
# Validacao
df_medalhas[df_medalhas.athlete_url == 'https://olympics.com/en/athletes/aaron-peirsol']

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code,game_season,game_year
5478,Swimming,beijing-2008,100m backstroke men,Men,GOLD,Athlete,NaN,https://olympics.com/en/athletes/aaron-peirsol,Aaron PEIRSOL,United States of America,US,USA,Summer,2008
5504,Swimming,beijing-2008,200m backstroke men,Men,SILVER,Athlete,NaN,https://olympics.com/en/athletes/aaron-peirsol,Aaron PEIRSOL,United States of America,US,USA,Summer,2008
6294,Swimming,athens-2004,100m backstroke men,Men,GOLD,Athlete,NaN,https://olympics.com/en/athletes/aaron-peirsol,Aaron PEIRSOL,United States of America,US,USA,Summer,2004
6327,Swimming,athens-2004,200m backstroke men,Men,GOLD,Athlete,NaN,https://olympics.com/en/athletes/aaron-peirsol,Aaron PEIRSOL,United States of America,US,USA,Summer,2004
8084,Swimming,sydney-2000,200m backstroke men,Men,SILVER,Athlete,NaN,https://olympics.com/en/athletes/aaron-peirsol,Aaron PEIRSOL,United States of America,US,USA,Summer,2000


Agrupar quantidade de medalhistas individuais da edição anterior que cada país tem por olimpíada

In [ ]:
df_medalhista_olimpíadas_ant_pais = df_medalhistas_omplipadas_ant.groupby(['slug_game','game_year','country_name','country_code','country_3_letter_code'])['Qtd_Medalhas_Individuais'].sum().reset_index(name = 'Qtd_Atletas_Medalhistas_Individuais_olimpíada_Anterior')
df_medalhista_olimpíadas_ant_pais.head()

,slug_game,game_year,country_name,country_code,country_3_letter_code,Qtd_Atletas_Medalhistas_Individuais_Olimpiada_Anterior
0,athens-2004,2004,Argentina,AR,ARG,1
1,athens-2004,2004,Australia,AU,AUS,32
2,athens-2004,2004,Austria,AT,AUT,6
3,athens-2004,2004,Azerbaijan,AZ,AZE,2
4,athens-2004,2004,Bahamas,BS,BAH,2


### Juntando informações de olimpíadas

In [ ]:
query = """
select
   a.*
  ,b.Qtd_Esportes_Distintos
  ,c.Qtd_times
  ,d.Qtd_Medalhas_Solo
  ,e.Qtd_Atletas_Medalhistas_Individuais_olimpíada_Anterior as Qtd_Medalhista_Ind_Ed_Ant
from df_medalhas_pais as a
left join df_qtd_esportes as b                     on a.slug_game = b.slug_game and a.country_3_letter_code	= b.country_3_letter_code
left join df_times as c                            on a.slug_game = c.slug_game and a.country_3_letter_code	= c.country_3_letter_code
left join df_medalhas_solo as d                    on a.slug_game = d.slug_game and a.country_3_letter_code	= d.country_3_letter_code
left join df_medalhista_olimpíadas_ant_pais as e   on a.slug_game = e.slug_game and a.country_3_letter_code	= e.country_3_letter_code


"""
df_treino_0 = psql.sqldf(query, locals())


In [ ]:
df_treino_0.head()

,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze,Qtd_Esportes_Distintos,Qtd_Times,Qtd_Medalhas_Solo,Qtd_Medalhista_Ind_Ed_Ant
0,athens-2004,Argentina,AR,ARG,2004,6,2,0,4,22,80,63,1.0
1,athens-2004,Australia,AU,AUS,2004,50,17,16,17,36,270,205,32.0
2,athens-2004,Austria,AT,AUT,2004,7,2,4,1,21,69,59,6.0
3,athens-2004,Azerbaijan,AZ,AZE,2004,5,1,0,4,9,32,32,2.0
4,athens-2004,Bahamas,BS,BAH,2004,2,1,0,1,3,14,11,2.0


In [ ]:
print(len(df_medalhas_pais))
print(len(df_treino_0))

504
504


In [ ]:
df_treino_0.game_year.unique()

array([2004, 2008, 2012, 2016, 2000, 2020])

In [ ]:
df_medalhista_olimpíadas_ant_pais.head()

,slug_game,game_year,country_name,country_code,country_3_letter_code,Qtd_Atletas_Medalhistas_Individuais_Olimpiada_Anterior
0,athens-2004,2004,Argentina,AR,ARG,1
1,athens-2004,2004,Australia,AU,AUS,32
2,athens-2004,2004,Austria,AT,AUT,6
3,athens-2004,2004,Azerbaijan,AZ,AZE,2
4,athens-2004,2004,Bahamas,BS,BAH,2


In [ ]:
df_medalhas_pais.head()

,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze
0,athens-2004,Argentina,AR,ARG,2004,6,2,0,4
1,athens-2004,Australia,AU,AUS,2004,50,17,16,17
2,athens-2004,Austria,AT,AUT,2004,7,2,4,1
3,athens-2004,Azerbaijan,AZ,AZE,2004,5,1,0,4
4,athens-2004,Bahamas,BS,BAH,2004,2,1,0,1


In [ ]:
df_medalhista_olimpíadas_ant_pais

,slug_game,game_year,country_name,country_code,country_3_letter_code,Qtd_Atletas_Medalhistas_Individuais_Olimpiada_Anterior
0,athens-2004,2004,Argentina,AR,ARG,1
1,athens-2004,2004,Australia,AU,AUS,32
2,athens-2004,2004,Austria,AT,AUT,6
3,athens-2004,2004,Azerbaijan,AZ,AZE,2
4,athens-2004,2004,Bahamas,BS,BAH,2
...,...,...,...,...,...,...
478,tokyo-2020,2020,Uganda,UG,UGA,4
479,tokyo-2020,2020,Ukraine,UA,UKR,15
480,tokyo-2020,2020,United States of America,US,USA,85
481,tokyo-2020,2020,Uzbekistan,UZ,UZB,5


### Tratando as informações socioeconômicas

In [ ]:
df_gov.head()

,Year,Country,Sector,Expenditure(million USD),GDP(%),LastYear
0,2006,Afghanistan,Education,324.499,4.56737,2017.0
1,2007,Afghanistan,Education,357.221,3.79531,2017.0
2,2008,Afghanistan,Education,451.464,4.41017,2017.0
3,2009,Afghanistan,Education,580.457,5.00604,2017.0
4,2010,Afghanistan,Education,824.899,5.61198,2017.0


In [ ]:
df_treino_0['Primeiro_ano_prep'] = df_treino_0['game_year'] - 3
df_treino_0['Segundo_ano_prep'] =  df_treino_0['game_year'] - 2
df_treino_0['Terceiro_ano_prep'] = df_treino_0['game_year'] - 1

In [ ]:
query = """
select
   a.*
  ,case when b.Primeiro_ano_prep  then b.slug_game
        when c.Segundo_ano_prep   then c.slug_game
        when d.Terceiro_ano_prep  then d.slug_game
  else null end as proximos_jogos
from df_gov as a
left join df_treino_0 as b on cast(a.year as int) = cast(b.Primeiro_ano_prep as int)
left join df_treino_0 as c on cast(a.year as int) = cast(c.Segundo_ano_prep  as int)
left join df_treino_0 as d on cast(a.year as int) = cast(d.Terceiro_ano_prep as int)
"""


In [ ]:
df_gov_2 = psql.sqldf(query, locals())

In [ ]:
df_gov_2.tail()

,Year,Country,Sector,Expenditure(million USD),GDP(%),LastYear,proximos_jogos
279420,2011,Yemen,"Recreation, culture and religion",155.753,0.50115,2012.0,london-2012
279421,2012,Yemen,"Recreation, culture and religion",166.196,0.51815,2012.0,None
279422,2021,Yemen,"Recreation, culture and religion",166.196,0.51815,NaN,None
279423,2022,Yemen,"Recreation, culture and religion",166.196,0.51815,NaN,None
279424,2023,Yemen,"Recreation, culture and religion",166.196,0.51815,NaN,None


In [ ]:
df_gov_2.proximos_jogos.unique()

array(['beijing-2008', None, 'london-2012', 'rio-2016', 'tokyo-2020',
       'athens-2004'], dtype=object)

In [ ]:
df_gov_2[df_gov_2['proximos_jogos'] == 'athens-2004']

,Year,Country,Sector,Expenditure(million USD),GDP(%),LastYear,proximos_jogos
6370,2001,Australia,Education,22339.428,5.71297,2021.0,athens-2004
6371,2001,Australia,Education,22339.428,5.71297,2021.0,athens-2004
6372,2001,Australia,Education,22339.428,5.71297,2021.0,athens-2004
6373,2001,Australia,Education,22339.428,5.71297,2021.0,athens-2004
6374,2001,Australia,Education,22339.428,5.71297,2021.0,athens-2004
...,...,...,...,...,...,...,...
278898,2003,Yemen,"Recreation, culture and religion",67.986,0.50154,2012.0,athens-2004
278899,2003,Yemen,"Recreation, culture and religion",67.986,0.50154,2012.0,athens-2004
278900,2003,Yemen,"Recreation, culture and religion",67.986,0.50154,2012.0,athens-2004
278901,2003,Yemen,"Recreation, culture and religion",67.986,0.50154,2012.0,athens-2004


In [ ]:
df_gov_2 = df_gov_2.groupby(['Country','Sector','proximos_jogos']).agg(
        media_gpd = ('GDP(%)','mean'),      # Calculando a média da coluna %GPD
        soma_Expediture = ('Expenditure(million USD)','sum')).reset_index()

In [ ]:
df_gov_2.tail()

,Country,Sector,proximos_jogos,media_gpd,soma_Expediture
1128,Yemen,Health,beijing-2008,1.235737,72120.651
1129,Yemen,Health,london-2012,1.351140,104029.375
1130,Yemen,"Recreation, culture and religion",athens-2004,0.464145,8898.574
1131,Yemen,"Recreation, culture and religion",beijing-2008,0.408937,24000.168
1132,Yemen,"Recreation, culture and religion",london-2012,0.419843,32319.890


In [ ]:
df_treino_0[df_treino_0['country_name'] == 'Uzbekistan'].head()

,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze,Qtd_Esportes_Distintos,Qtd_Times,Qtd_Medalhas_Solo,Qtd_Medalhista_Ind_Ed_Ant,Primeiro_ano_prep,Segundo_ano_prep,Terceiro_ano_prep
71,athens-2004,Uzbekistan,UZ,UZB,2004,5,2,1,2,10,48,48,3.0,2001,2002,2003
157,beijing-2008,Uzbekistan,UZ,UZB,2008,4,0,1,3,14,50,50,4.0,2005,2006,2007
244,london-2012,Uzbekistan,UZ,UZB,2012,2,0,0,2,13,49,49,2.0,2009,2010,2011
329,rio-2016,Uzbekistan,UZ,UZB,2016,13,4,2,7,14,57,55,12.0,2013,2014,2015
410,sydney-2000,Uzbekistan,UZ,UZB,2000,4,1,1,2,10,47,46,3.0,1997,1998,1999


In [ ]:
# Trazendo dados de investimentos para o dataframe de treino

query  = """
select
   a.*
  ,b.soma_Expediture as Health_Expenditure
  ,b.media_gpd       as Health_Media_Gpd
  ,c.soma_Expediture as Culture_Expenditure
  ,c.media_gpd       as Culture_Media_Gpd
  ,d.soma_Expediture as Education_Expenditure
  ,d.media_gpd       as Education_Media_Gpd
from df_treino_0 as a
left join df_gov_2 as b on a.slug_game = b.proximos_jogos and a.country_name = b.Country and b.Sector = 'Health'
left join df_gov_2 as c on a.slug_game = c.proximos_jogos and a.country_name = c.Country and c.Sector = 'Recreation, culture and religion'
left join df_gov_2 as d on a.slug_game = d.proximos_jogos and a.country_name = d.Country and d.Sector = 'Education'

"""

In [ ]:
df_treino_1 = psql.sqldf(query, locals())

In [ ]:
df_treino_1.head()

,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze,Qtd_Esportes_Distintos,Qtd_Times,Qtd_Medalhas_Solo,Qtd_Medalhista_Ind_Ed_Ant,Primeiro_ano_prep,Segundo_ano_prep,Terceiro_ano_prep,Health_Expenditure,Health_Media_Gpd,Culture_Expenditure,Culture_Media_Gpd,Education_Expenditure,Education_Media_Gpd
0,athens-2004,Argentina,AR,ARG,2004,6,2,0,4,22,80,63,1.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
1,athens-2004,Australia,AU,AUS,2004,50,17,16,17,36,270,205,32.0,2001,2002,2003,6226633.874,6.062443,971555.250,0.944507,5797655.578,5.654797
2,athens-2004,Austria,AT,AUT,2004,7,2,4,1,21,69,59,6.0,2001,2002,2003,3626783.512,7.272650,686135.252,1.380663,2597898.168,5.216793
3,athens-2004,Azerbaijan,AZ,AZE,2004,5,1,0,4,9,32,32,2.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
4,athens-2004,Bahamas,BS,BAH,2004,2,1,0,1,3,14,11,2.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(len(df_treino_0))
print(len(df_treino_1))

504
504


In [ ]:
df_gov_2[df_gov_2['Country'] == 'Australia']

,Country,Sector,proximos_jogos,media_gpd,soma_Expediture
30,Australia,Education,athens-2004,5.654797,5.797656e+06
31,Australia,Education,beijing-2008,5.423863,1.210149e+07
32,Australia,Education,london-2012,6.315263,2.057888e+07
33,Australia,Education,rio-2016,5.809150,2.122982e+07
34,Australia,Education,tokyo-2020,6.041383,2.356597e+07
35,Australia,Health,athens-2004,6.062443,6.226634e+06
36,Australia,Health,beijing-2008,6.335287,1.416491e+07
37,Australia,Health,london-2012,6.958007,2.293138e+07
38,Australia,Health,rio-2016,7.235900,2.640072e+07
39,Australia,Health,tokyo-2020,7.623493,2.974210e+07


In [ ]:
import numpy as np
#df_treino_1['country_name'][df_treino_1['Health_Expenditure'].isna()].unique()

df_treino_1[df_treino_1['Health_Expenditure'].isna()]

,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze,Qtd_Esportes_Distintos,Qtd_Times,Qtd_Medalhas_Solo,Qtd_Medalhista_Ind_Ed_Ant,Primeiro_ano_prep,Segundo_ano_prep,Terceiro_ano_prep,Health_Expenditure,Health_Media_Gpd,Culture_Expenditure,Culture_Media_Gpd,Education_Expenditure,Education_Media_Gpd
0,athens-2004,Argentina,AR,ARG,2004,6,2,0,4,22,80,63,1.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
3,athens-2004,Azerbaijan,AZ,AZE,2004,5,1,0,4,9,32,32,2.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
4,athens-2004,Bahamas,BS,BAH,2004,2,1,0,1,3,14,11,2.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
9,athens-2004,Cameroon,CM,CMR,2004,1,1,0,0,4,8,8,1.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
10,athens-2004,Canada,CA,CAN,2004,12,3,6,3,33,176,137,9.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,tokyo-2020,Tunisia,TN,TUN,2020,2,1,1,0,16,48,40,2.0,2017,2018,2019,NaN,NaN,NaN,NaN,NaN,NaN
497,tokyo-2020,Turkey,TR,TUR,2020,13,2,2,9,17,78,74,13.0,2017,2018,2019,NaN,NaN,NaN,NaN,NaN,NaN
498,tokyo-2020,Turkmenistan,TM,TKM,2020,1,0,1,0,2,5,5,1.0,2017,2018,2019,NaN,NaN,NaN,NaN,NaN,NaN
499,tokyo-2020,Uganda,UG,UGA,2020,4,2,1,1,3,18,18,4.0,2017,2018,2019,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_treino_1[df_treino_1['country_name']=='Canada']

,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze,Qtd_Esportes_Distintos,Qtd_Times,Qtd_Medalhas_Solo,Qtd_Medalhista_Ind_Ed_Ant,Primeiro_ano_prep,Segundo_ano_prep,Terceiro_ano_prep,Health_Expenditure,Health_Media_Gpd,Culture_Expenditure,Culture_Media_Gpd,Education_Expenditure,Education_Media_Gpd
10,athens-2004,Canada,CA,CAN,2004,12,3,6,3,33,176,137,9.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
87,beijing-2008,Canada,CA,CAN,2008,20,3,9,8,34,203,162,15.0,2005,2006,2007,NaN,NaN,NaN,NaN,NaN,NaN
175,london-2012,Canada,CA,CAN,2012,18,2,5,11,35,202,164,12.0,2009,2010,2011,3.392022e+07,8.352170,3652344.600,0.899877,1.952212e+07,4.807617
259,rio-2016,Canada,CA,CAN,2016,22,4,3,15,38,201,163,14.0,2013,2014,2015,3.638256e+07,8.132717,3835761.078,0.857307,2.060672e+07,4.601887
345,sydney-2000,Canada,CA,CAN,2000,14,3,3,8,32,204,162,10.0,1997,1998,1999,NaN,NaN,NaN,NaN,NaN,NaN
426,tokyo-2020,Canada,CA,CAN,2020,24,7,6,11,40,205,152,15.0,2017,2018,2019,3.834195e+07,8.152083,4049513.400,0.861010,2.235494e+07,4.752220


In [ ]:
df_gov_2[df_gov_2['Country'] == 'Canada']

,Country,Sector,proximos_jogos,media_gpd,soma_Expediture
135,Canada,Education,london-2012,4.807617,1.952212e+07
136,Canada,Education,rio-2016,4.601887,2.060672e+07
137,Canada,Education,tokyo-2020,4.752220,2.235494e+07
138,Canada,Health,london-2012,8.352170,3.392022e+07
139,Canada,Health,rio-2016,8.132717,3.638256e+07
140,Canada,Health,tokyo-2020,8.152083,3.834195e+07
141,Canada,"Recreation, culture and religion",london-2012,0.899877,3.652345e+06
142,Canada,"Recreation, culture and religion",rio-2016,0.857307,3.835761e+06
143,Canada,"Recreation, culture and religion",tokyo-2020,0.861010,4.049513e+06


In [ ]:
df_gov[df_gov['Country'] == 'Canada']

,Year,Country,Sector,Expenditure(million USD),GDP(%),LastYear
621,2008,Canada,Education,67800.475,4.36596,2021.0
622,2009,Canada,Education,67248.900,4.89217,2021.0
623,2010,Canada,Education,77803.135,4.81078,2021.0
624,2011,Canada,Education,84619.974,4.71990,2021.0
625,2012,Canada,Education,85797.503,4.69176,2021.0
626,2013,Canada,Education,85945.820,4.65275,2021.0
627,2014,Canada,Education,81910.147,4.54165,2021.0
628,2015,Canada,Education,71757.108,4.61126,2021.0
629,2016,Canada,Education,71258.992,4.66280,2021.0
630,2017,Canada,Education,78103.244,4.73491,2021.0


## Dados de treino olimpíadas de 2024

### Qtd Medalhas

In [ ]:
df_medalhas_2024.head()

,country_code,country,country_long,Gold Medal,Silver Medal,Bronze Medal,Total
0,USA,United States,United States of America,40,44,42,126
1,CHN,China,People's Republic of China,40,27,24,91
2,JPN,Japan,Japan,20,12,13,45
3,AUS,Australia,Australia,18,19,16,53
4,FRA,France,France,16,26,22,64


In [ ]:
df_medalhas_2024['slug_game'] = 'paris-2024'
df_medalhas_2024['year'] = 2024

### Qtd Times

In [ ]:
df_resultados_2024 = df_resultados_2024[['event_name','discipline_name','participant_country','participant_country_code']]
print(len(df_resultados_2024))
df_resultados_2024.drop_duplicates(inplace=True)
print(len(df_resultados_2024))

21316
6592


In [ ]:
df_qtd_times_2024 = df_resultados_2024.groupby(['participant_country', 'participant_country_code']).size().reset_index(name='qtd_times')

In [ ]:
df_qtd_times_2024['slug_game'] = 'paris-2024'
df_qtd_times_2024['year'] = 2024

In [ ]:
df_qtd_times_2024.head()

,participant_country,participant_country_code,qtd_times,slug_game,year
0,AIN,AIN,30,paris-2024,2024
1,Afghanistan,AFG,6,paris-2024,2024
2,Albania,ALB,9,paris-2024,2024
3,Algeria,ALG,43,paris-2024,2024
4,American Samoa,ASA,2,paris-2024,2024


### Qtd Esportes Distintos

In [ ]:
df_resultados_2024.columns

Index(['event_name', 'discipline_name', 'participant_country',
       'participant_country_code'],
      dtype='object')

In [ ]:
df_esportes_2024 = df_resultados_2024[['discipline_name','participant_country','participant_country_code']]
df_esportes_2024.drop_duplicates(inplace=True)

<ipython-input-111-2b4bf1070908>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_esportes_2024.drop_duplicates(inplace=True)


In [ ]:
df_esportes_2024 = df_esportes_2024.groupby(['participant_country','participant_country_code']).size().reset_index(name = 'Qtd_Esportes_Distintos')

In [ ]:
df_esportes_2024['slug_game'] = 'paris-2024'
df_esportes_2024['year'] = 2024

In [ ]:
df_esportes_2024.head(2)

,participant_country,participant_country_code,Qtd_Esportes_Distintos,slug_game,year
0,AIN,AIN,10,paris-2024,2024
1,Afghanistan,AFG,4,paris-2024,2024


### Juntando informações

In [ ]:
query = """


select
  a.*
  ,b.qtd_times
  ,c.Qtd_Esportes_Distintos
from df_medalhas_2024 as a
left join df_qtd_times_2024 as b  on a.slug_game = b.slug_game and a.country_code =  b.participant_country_code
left join df_esportes_2024 as c   on a.slug_game = c.slug_game and a.country_code =  c.participant_country_code

"""

df_2024 = psql.sqldf(query, locals())

In [ ]:
print(len(df_2024))
print(len(df_medalhas_2024))

92
92


### Informações socioeconômicas

In [ ]:
df_2024['Primeiro_ano_prep'] = df_2024['year'] - 3
df_2024['Segundo_ano_prep'] =  df_2024['year'] - 2
df_2024['Terceiro_ano_prep'] = df_2024['year'] - 1

In [ ]:
query = """
select
   a.*
  ,case when b.Primeiro_ano_prep  then b.slug_game
        when c.Segundo_ano_prep   then c.slug_game
        when d.Terceiro_ano_prep  then d.slug_game
  else null end as proximos_jogos
from df_gov as a
left join df_2024 as b on cast(a.year as int) = cast(b.Primeiro_ano_prep as int)
left join df_2024 as c on cast(a.year as int) = cast(c.Segundo_ano_prep  as int)
left join df_2024 as d on cast(a.year as int) = cast(d.Terceiro_ano_prep as int)
"""

In [ ]:
df_gov_2024 = psql.sqldf(query, locals())
print(len(df_gov_2024))
print(len(df_gov))

83726
5375


In [ ]:
df_gov_2024 = df_gov_2024.groupby(['Country','Sector','proximos_jogos']).agg(
        media_gpd = ('GDP(%)','mean'),      # Calculando a média da coluna %GPD
        soma_Expediture = ('Expenditure(million USD)','sum')).reset_index()

In [ ]:
print(len(df_gov_2024))
print(len(df_gov))

287
5375


In [ ]:
df_gov_2024[df_gov_2024.proximos_jogos == 'paris-2024']

,Country,Sector,proximos_jogos,media_gpd,soma_Expediture
0,Afghanistan,Education,paris-2024,3.94815,205911.732
1,Afghanistan,Health,paris-2024,2.14894,112075.596
2,Afghanistan,"Recreation, culture and religion",paris-2024,0.31900,16637.280
3,Albania,Education,paris-2024,3.08970,155713.956
4,Albania,Health,paris-2024,3.36297,169486.080
...,...,...,...,...,...
282,Vanuatu,Health,paris-2024,2.67744,6759.516
283,Vanuatu,"Recreation, culture and religion",paris-2024,0.30565,771.696
284,Yemen,Education,paris-2024,6.03021,533832.576
285,Yemen,Health,paris-2024,1.76964,156659.532


In [ ]:
sorted(df_gov_2024.Country.unique())

['Afghanistan',
 'Albania',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Belarus',
 'Belgium',
 'Bolivia (Plurinational State of)',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'China',
 'China, Hong Kong SAR',
 'China, Macao SAR',
 'Colombia',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Fiji',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Guatemala',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran (Islamic Republic of)',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Kazakhstan',
 'Kenya',
 'Kiribati',
 'Korea, Republic of',
 'Kuwait',
 'Kyrgyzstan',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Maldives',
 'Malta',
 'Marshall Islands',
 'Mauritius',
 'Moldova, Republic of',
 'Mongolia',
 'Myanmar',
 'Nauru',
 'Nepal',
 'Netherlands (Kingdom of the)',
 'New Zealand',
 'Norway',
 'Panama',
 'Paraguay',
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',
 'Romania',
 'Russian F

In [ ]:
sorted(df_2024.country_long.unique())

['AIN',
 'Albania',
 'Algeria',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Belgium',
 'Botswana',
 'Brazil',
 'Bulgaria',
 'Cabo Verde',
 'Canada',
 'Chile',
 'Chinese Taipei',
 'Colombia',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 "Democratic People's Republic of Korea",
 'Denmark',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'Ethiopia',
 'Fiji',
 'France',
 'Georgia',
 'Germany',
 'Great Britain',
 'Greece',
 'Grenada',
 'Guatemala',
 'Hong Kong, China',
 'Hungary',
 'India',
 'Indonesia',
 'Ireland',
 'Islamic Republic of Iran',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kosovo',
 'Kyrgyzstan',
 'Lithuania',
 'Malaysia',
 'Mexico',
 'Mongolia',
 'Morocco',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Pakistan',
 'Panama',
 "People's Republic of China",
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',
 'Puerto Rico',
 'Qatar',
 'Refugee Olympic Team',
 'Republic of Korea',


In [ ]:
df_2024['country_long'] = df_2024['country_long'].replace("People's Republic of China", "China")

In [ ]:
df_2024.head()

,country_code,country,country_long,Gold Medal,Silver Medal,Bronze Medal,Total,slug_game,year,qtd_times,Qtd_Esportes_Distintos,Primeiro_ano_prep,Segundo_ano_prep,Terceiro_ano_prep
0,USA,United States,United States of America,40,44,42,126,paris-2024,2024,264,44,2021,2022,2023
1,CHN,China,China,40,27,24,91,paris-2024,2024,232,42,2021,2022,2023
2,JPN,Japan,Japan,20,12,13,45,paris-2024,2024,193,41,2021,2022,2023
3,AUS,Australia,Australia,18,19,16,53,paris-2024,2024,220,42,2021,2022,2023
4,FRA,France,France,16,26,22,64,paris-2024,2024,259,45,2021,2022,2023


### Juntando informações

In [ ]:
# Trazendo dados de investimentos para o dataframe de treino

query  = """
select
   a.*
  ,b.soma_Expediture as Health_Expenditure
  ,b.media_gpd       as Health_Media_Gpd
  ,c.soma_Expediture as Culture_Expenditure
  ,c.media_gpd       as Culture_Media_Gpd
  ,d.soma_Expediture as Education_Expenditure
  ,d.media_gpd       as Education_Media_Gpd
from df_2024 as a
left join df_gov_2024 as b on a.slug_game = b.proximos_jogos and lower(a.country_long) = lower(b.Country) and b.Sector = 'Health'
left join df_gov_2024 as c on a.slug_game = c.proximos_jogos and lower(a.country_long) = lower(c.Country) and c.Sector = 'Recreation, culture and religion'
left join df_gov_2024 as d on a.slug_game = d.proximos_jogos and lower(a.country_long) = lower(d.Country) and d.Sector = 'Education'

"""

df_2024_2 = psql.sqldf(query, locals())

In [ ]:
print(len(df_2024))
print(len(df_2024_2))

92
92


## Juntanto informações de 2004 a 2020 com 2024

In [ ]:
df_2024_2.head()

,country_code,country,country_long,Gold Medal,Silver Medal,Bronze Medal,Total,slug_game,year,qtd_times,Qtd_Esportes_Distintos,Primeiro_ano_prep,Segundo_ano_prep,Terceiro_ano_prep,Health_Expenditure,Health_Media_Gpd,Culture_Expenditure,Culture_Media_Gpd,Education_Expenditure,Education_Media_Gpd
0,USA,United States,United States of America,40,44,42,126,paris-2024,2024,264,44,2021,2022,2023,6.622703e+08,10.24494,1.725794e+07,0.26697,3.618440e+08,5.59752
1,CHN,China,China,40,27,24,91,paris-2024,2024,232,42,2021,2022,2023,1.173621e+08,2.89510,1.702467e+07,0.41997,1.465649e+08,3.61548
2,JPN,Japan,Japan,20,12,13,45,paris-2024,2024,193,41,2021,2022,2023,1.270468e+08,9.31645,6.217111e+06,0.45591,4.778849e+07,3.50437
3,AUS,Australia,Australia,18,19,16,53,paris-2024,2024,220,42,2021,2022,2023,3.936536e+07,8.22286,4.415049e+06,0.92224,2.773219e+07,5.79286
4,FRA,France,France,16,26,22,64,paris-2024,2024,259,45,2021,2022,2023,7.523770e+07,9.18937,1.149774e+07,1.40431,4.278405e+07,5.22555


In [ ]:
df_treino_1.head(2)

,slug_game,country_name,country_code,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze,Qtd_Esportes_Distintos,Qtd_Times,Qtd_Medalhas_Solo,Qtd_Medalhista_Ind_Ed_Ant,Primeiro_ano_prep,Segundo_ano_prep,Terceiro_ano_prep,Health_Expenditure,Health_Media_Gpd,Culture_Expenditure,Culture_Media_Gpd,Education_Expenditure,Education_Media_Gpd
0,athens-2004,Argentina,AR,ARG,2004,6,2,0,4,22,80,63,1.0,2001,2002,2003,NaN,NaN,NaN,NaN,NaN,NaN
1,athens-2004,Australia,AU,AUS,2004,50,17,16,17,36,270,205,32.0,2001,2002,2003,6226633.874,6.062443,971555.25,0.944507,5797655.578,5.654797


In [ ]:
df_treino_1.drop(columns = {'Primeiro_ano_prep', 'Segundo_ano_prep',
       'Terceiro_ano_prep'},inplace=True)

In [ ]:
df_treino_1.drop(columns = {'country_code'},inplace=True)

In [ ]:
df_2024_2.columns

Index(['country_code', 'country', 'country_long', 'Gold Medal', 'Silver Medal',
       'Bronze Medal', 'Total', 'slug_game', 'year', 'qtd_times',
       'Qtd_Esportes_Distintos', 'Primeiro_ano_prep', 'Segundo_ano_prep',
       'Terceiro_ano_prep', 'Health_Expenditure', 'Health_Media_Gpd',
       'Culture_Expenditure', 'Culture_Media_Gpd', 'Education_Expenditure',
       'Education_Media_Gpd'],
      dtype='object')

In [ ]:
df_2024_2.drop(columns = {'Primeiro_ano_prep', 'Segundo_ano_prep',
       'Terceiro_ano_prep'},inplace=True)

In [ ]:
# Renomear colunas diretamente no DataFrame original
df_2024_2.rename(columns={'country_code': 'country_3_letter_code', 'country_long': 'country_name', 'year':'game_year','Gold Medal': 'Qtd_Medalhas_Ouro',
                          'Silver Medal':'Qtd_Medalhas_Prata','Bronze Medal':'Qtd_Medalhas_Bronze','Total':'Qtd_Medalhas_Pais'
                          }, inplace=True)

In [ ]:
df_2024_2.rename(columns={'qtd_times':'Qtd_Times'
                          }, inplace=True)

In [ ]:
df_2024_2.columns

Index(['country_3_letter_code', 'country', 'country_name', 'Qtd_Medalhas_Ouro',
       'Qtd_Medalhas_Prata', 'Qtd_Medalhas_Bronze', 'Qtd_Medalhas_Pais',
       'slug_game', 'game_year', 'Qtd_Times', 'Qtd_Esportes_Distintos',
       'Health_Expenditure', 'Health_Media_Gpd', 'Culture_Expenditure',
       'Culture_Media_Gpd', 'Education_Expenditure', 'Education_Media_Gpd'],
      dtype='object')

In [ ]:
df_treino_1.columns

Index(['slug_game', 'country_name', 'country_3_letter_code', 'game_year',
       'Qtd_Medalhas_Pais', 'Qtd_Medalhas_Ouro', 'Qtd_Medalhas_Prata',
       'Qtd_Medalhas_Bronze', 'Qtd_Esportes_Distintos', 'Qtd_Times',
       'Qtd_Medalhas_Solo', 'Qtd_Medalhista_Ind_Ed_Ant', 'Health_Expenditure',
       'Health_Media_Gpd', 'Culture_Expenditure', 'Culture_Media_Gpd',
       'Education_Expenditure', 'Education_Media_Gpd'],
      dtype='object')

In [ ]:
#Reordenando colunas do dataframe de 2024
new_order = ['slug_game', 'country_name', 'country_3_letter_code', 'game_year',
       'Qtd_Medalhas_Pais', 'Qtd_Medalhas_Ouro', 'Qtd_Medalhas_Prata',
       'Qtd_Medalhas_Bronze', 'Qtd_Esportes_Distintos', 'Qtd_Times',
        'Health_Expenditure',
       'Health_Media_Gpd', 'Culture_Expenditure', 'Culture_Media_Gpd',
       'Education_Expenditure', 'Education_Media_Gpd']

In [ ]:
df_2024_3  = df_2024_2[new_order]

In [ ]:
df_treino_1.drop(columns = {'Qtd_Medalhas_Solo', 'Qtd_Medalhista_Ind_Ed_Ant'},inplace = True)

In [ ]:
df_treino_1.columns

Index(['slug_game', 'country_name', 'country_3_letter_code', 'game_year',
       'Qtd_Medalhas_Pais', 'Qtd_Medalhas_Ouro', 'Qtd_Medalhas_Prata',
       'Qtd_Medalhas_Bronze', 'Qtd_Esportes_Distintos', 'Qtd_Times',
       'Health_Expenditure', 'Health_Media_Gpd', 'Culture_Expenditure',
       'Culture_Media_Gpd', 'Education_Expenditure', 'Education_Media_Gpd'],
      dtype='object')

In [ ]:
df_2004_2024 = pd.concat([df_2024_3, df_treino_1], axis=0, ignore_index=True)

In [ ]:
df_2004_2024.head()

,slug_game,country_name,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze,Qtd_Esportes_Distintos,Qtd_Times,Health_Expenditure,Health_Media_Gpd,Culture_Expenditure,Culture_Media_Gpd,Education_Expenditure,Education_Media_Gpd
0,paris-2024,United States of America,USA,2024,126,40,44,42,44,264,6.622703e+08,10.24494,1.725794e+07,0.26697,3.618440e+08,5.59752
1,paris-2024,China,CHN,2024,91,40,27,24,42,232,1.173621e+08,2.89510,1.702467e+07,0.41997,1.465649e+08,3.61548
2,paris-2024,Japan,JPN,2024,45,20,12,13,41,193,1.270468e+08,9.31645,6.217111e+06,0.45591,4.778849e+07,3.50437
3,paris-2024,Australia,AUS,2024,53,18,19,16,42,220,3.936536e+07,8.22286,4.415049e+06,0.92224,2.773219e+07,5.79286
4,paris-2024,France,FRA,2024,64,16,26,22,45,259,7.523770e+07,9.18937,1.149774e+07,1.40431,4.278405e+07,5.22555


In [ ]:
df_2004_2024.tail()

,slug_game,country_name,country_3_letter_code,game_year,Qtd_Medalhas_Pais,Qtd_Medalhas_Ouro,Qtd_Medalhas_Prata,Qtd_Medalhas_Bronze,Qtd_Esportes_Distintos,Qtd_Times,Health_Expenditure,Health_Media_Gpd,Culture_Expenditure,Culture_Media_Gpd,Education_Expenditure,Education_Media_Gpd
591,tokyo-2020,Uganda,UGA,2020,4,2,1,1,3,18,NaN,NaN,NaN,NaN,NaN,NaN
592,tokyo-2020,Ukraine,UKR,2020,19,1,6,12,26,126,1.208774e+06,3.32023,2.945715e+05,0.808047,2.175703e+06,5.956783
593,tokyo-2020,United States of America,USA,2020,113,39,41,33,43,409,5.323151e+08,9.37481,1.555373e+07,0.273987,3.369399e+08,5.937970
594,tokyo-2020,Uzbekistan,UZB,2020,5,3,0,2,13,54,3.635745e+05,2.26272,6.816400e+04,0.424230,9.134292e+05,5.671833
595,tokyo-2020,Venezuela,VEN,2020,4,1,3,0,15,27,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_2004_2024.to_csv('df_olimpíadas_2004_2024.csv', sep = ';',index = False)

## Dados para análise de sentimento

No nosso projeto também iremos usar um LLM para analisar a filosofia dos atletas.

In [ ]:
df_atletas2024.head(2)

,code,current,name,name_short,name_tv,gender,function,country_code,country,country_long,nationality,nationality_long,nationality_code,height,weight,disciplines,events,birth_date,birth_place,birth_country,residence_place,residence_country,nickname,hobbies,occupation,education,family,lang,coach,reason,hero,influence,philosophy,sporting_relatives,ritual,other_sports
0,1532872,True,ALEKSANYAN Artur,ALEKSANYAN A,Artur ALEKSANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0.0,0.0,['Wrestling'],"[""Men's Greco-Roman 97kg""]",1991-10-21,GYUMRI,Armenia,GYUMRI,Armenia,White Bear,Playing and watching football,Athlete,Graduated from Shirak State University (Gyumri...,"Father, Gevorg Aleksanyan","Armenian, English, Russian","Gevorg Aleksanyan (ARM), father",He followed his father and his uncle into the ...,"Footballer Zinedine Zidane (FRA), World Cup wi...","His father, Gevorg Aleksanyan","""Wrestling is my life."" (mediamax.am. 18 May 2...",NaN,NaN,NaN
1,1532873,True,AMOYAN Malkhas,AMOYAN M,Malkhas AMOYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0.0,0.0,['Wrestling'],"[""Men's Greco-Roman 77kg""]",1999-01-22,YEREVAN,Armenia,YEREVAN,Armenia,NaN,NaN,NaN,NaN,NaN,Armenian,NaN,NaN,NaN,NaN,"""To become a good athlete, you first have to b...","Uncle, Roman Amoyan (wrestling), 2008 Olympic ...",NaN,NaN


In [ ]:
df_atletas2024.tail(2)

,code,current,name,name_short,name_tv,gender,function,country_code,country,country_long,nationality,nationality_long,nationality_code,height,weight,disciplines,events,birth_date,birth_place,birth_country,residence_place,residence_country,nickname,hobbies,occupation,education,family,lang,coach,reason,hero,influence,philosophy,sporting_relatives,ritual,other_sports
11111,1899865,False,STAUT Kevin,NaN,NaN,Male,Athlete,FRA,France,France,France,France,FRA,NaN,NaN,[Equestrian],[Jumping Team],1980-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11112,1924402,False,CARVELL Charlie,NaN,NaN,Male,Athlete,GBR,Great Britain,Great Britain,Great Britain,Great Britain,GBR,NaN,NaN,[Athletics],[Men's 4 x 400m Relay],2004-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
null_percentage = df_atletas2024['philosophy'].isna().mean() * 100
print(f"A porcentagem de valores nulos na coluna 'philosophy' é {null_percentage:.2f}%")

A porcentagem de valores nulos na coluna 'philosophy' é 74.96%


In [ ]:
column_name = 'philosophy'  # Substitua pelo nome da coluna desejada

# Calcular a quantidade de valores não nulos na coluna
non_null_count = df_atletas2024[column_name].notna().sum()

print(f"A quantidade de valores não nulos na coluna '{column_name}' é {non_null_count}")

A quantidade de valores não nulos na coluna 'philosophy' é 2783


In [ ]:
df_analise_filosofia = df_atletas2024[['name','country_code','country','hobbies','education','hero','influence','philosophy']]
df_analise_filosofia['slug_game'] = 'paris-2024'
df_analise_filosofia['game_year'] = '2024'

<ipython-input-229-ead05eb8c896>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analise_filosofia['slug_game'] = 'paris-2024'
<ipython-input-229-ead05eb8c896>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analise_filosofia['game_year'] = '2024'


In [ ]:
df_analise_filosofia

,name,country_code,country,hobbies,education,hero,influence,philosophy,slug_game,game_year
0,ALEKSANYAN Artur,ARM,Armenia,Playing and watching football,Graduated from Shirak State University (Gyumri...,"Footballer Zinedine Zidane (FRA), World Cup wi...","His father, Gevorg Aleksanyan","""Wrestling is my life."" (mediamax.am. 18 May 2...",paris-2024,2024
1,AMOYAN Malkhas,ARM,Armenia,NaN,NaN,NaN,NaN,"""To become a good athlete, you first have to b...",paris-2024,2024
2,GALSTYAN Slavik,ARM,Armenia,NaN,NaN,NaN,NaN,NaN,paris-2024,2024
3,HARUTYUNYAN Arsen,ARM,Armenia,NaN,Graduated with a Master's degree from the Arme...,"Wrestler Armen Nazaryan (ARM, BUL), two-time O...",NaN,"“Nothing is impossible, set goals in front of ...",paris-2024,2024
4,TEVANYAN Vazgen,ARM,Armenia,NaN,Studied at the Armenian State Institute of Phy...,NaN,NaN,NaN,paris-2024,2024
...,...,...,...,...,...,...,...,...,...,...
11108,ADA ETO Sefora,GEQ,Equatorial Guinea,NaN,NaN,NaN,NaN,NaN,paris-2024,2024
11109,LIUZZI Emanuela,ITA,Italy,NaN,NaN,NaN,NaN,NaN,paris-2024,2024
11110,BOERS Isayah,NED,Netherlands,NaN,NaN,NaN,NaN,NaN,paris-2024,2024
11111,STAUT Kevin,FRA,France,NaN,NaN,NaN,NaN,NaN,paris-2024,2024


In [ ]:
df_analise_filosofia_cleaned = df_analise_filosofia.dropna(subset=['hobbies','education','hero','influence','philosophy'], how='any')

In [ ]:
len(df_analise_filosofia_cleaned)

462

### Exportação para um csv

In [ ]:
df_analise_filosofia_cleaned.to_csv('df_filosofia_paris_2024.csv',sep = ';')